# Swiss Job Market Demand Analysis – Data Cleaning

- Clean and standardize the raw job postings dataset  
- Translate job titles to English for consistency  
- Output: `Swiss-jobs-cleaned-final.csv` with 13 structured columns  

*Author: Vijai varma — Project run on 26/09/2025*


## Step 1: Imports and configuration
- Import core libraries for cleaning and translation  
- Define raw input and cleaned output file paths  
- Store Swiss canton abbreviations for parsing


In [28]:
import pandas as pd
import re
import unicodedata
from deep_translator import GoogleTranslator

# input and output file paths
RAW_PATH = "../data/Swiss-jobs-raw.csv"
OUT_CSV = "../data/Swiss-jobs-cleaned-final.csv"

# Swiss canton abbreviations
CANTONS = ["ZH","BE","LU","UR","SZ","OW","NW","GL","ZG","FR","SO","BS","BL",
           "SH","AR","AI","SG","GR","AG","TG","TI","VD","VS","NE","GE","JU"]

translator = GoogleTranslator(source="auto", target="en")


## Step 2: Helper functions
- Fix encoding issues and normalize text  
- Clean job titles, company names, and locations  
- Translate job titles into English  
- Extract job seniority from keywords


In [29]:
def normalize_text(s):
    if pd.isna(s): return ""
    s = str(s)
    try:
        if "Ã" in s:
            s = s.encode("latin1", "ignore").decode("utf-8", "ignore")
    except Exception:
        pass
    return unicodedata.normalize("NFKC", s).strip()

def clean_title(t):
    t = normalize_text(t).lower()
    t = re.sub(r"\([^)]*\)", "", t)
    t = re.sub(r"\d{2,}%|\d+/\d+|\b\d{4}\b", "", t)
    t = re.sub(r"[-–—|:;/•·•,]+", " ", t)
    return re.sub(r"\s+", " ", t).strip()

def translate_title(t):
    if not t or str(t).strip() == "":
        return t
    try:
        return translator.translate(str(t))
    except Exception:
        return t

def clean_company(c):
    return normalize_text(c)

def clean_location(l):
    l = normalize_text(l)
    remote_type = "Onsite"
    if "remote" in l.lower():
        remote_type = "Remote"
        l = l.replace("Remote in", "").replace("remote in", "").replace("Remote", "").strip()
    if "hybrid" in l.lower():
        remote_type = "Hybrid"
        l = re.sub(r"hybrid work in", "", l, flags=re.I).strip()
    l = re.sub(r"\b\d{4}\b", "", l)
    l = re.sub(r"\b\d+/\d+\b", "", l)
    parts = [p.strip() for p in l.split(",") if p.strip()]
    city = normalize_text(parts[0]).title() if parts else "Unknown"
    city = city.replace("ü","u").replace("ö","o").replace("ä","a").replace("ß","ss")
    canton = parts[1].upper() if len(parts) > 1 and parts[1].upper() in CANTONS else "Unknown"
    return city, canton, remote_type

def extract_seniority(title):
    if pd.isna(title): return "Unknown"
    title = title.lower()
    if any(x in title for x in ["intern","trainee","graduate"]): return "Entry/Intern"
    if "junior" in title: return "Junior"
    if any(x in title for x in ["senior","lead"]): return "Senior"
    if any(x in title for x in ["manager","head"]): return "Manager"
    return "Mid-level"


## Step 3: Load raw dataset
- Import the raw CSV file into Pandas  
- Preview dataset shape and first few rows


In [30]:
jobs_df = pd.read_csv(RAW_PATH)
print("Raw shape:", jobs_df.shape)
jobs_df.head(5)


Raw shape: (130, 6)


,job_id,title,company,location,date_posted,link
0,ace6dabef8894c7b,Reference Data Analyst,Eclerx,"Zürich, ZH",NaN,https://ch.indeed.com/rc/clk?jk=ace6dabef8894c...
1,c1ccef535fa878f6,Innosight Consulting Analyst - Switzerland,Huron Consulting Group Inc.,"Zürich, ZH",NaN,https://ch.indeed.com/rc/clk?jk=c1ccef535fa878...
2,9f657f8aca704ced,Investment Data Reporting Analyst (Fixed Term ...,LGT Capital Partners,"Pfäffikon, ZH",NaN,https://ch.indeed.com/rc/clk?jk=9f657f8aca704c...
3,10d2123eebee710b,Junior Financial Planning Analyst 80% - 100% (...,Dormakaba Group,"Rümlang, ZH",NaN,https://ch.indeed.com/rc/clk?jk=10d2123eebee71...
4,9e554eb40fad3074,"Junior Financial Data Analyst Graduate, Zurich...",Swiss Re,"Zürich, ZH",NaN,https://ch.indeed.com/rc/clk?jk=9e554eb40fad30...


## Step 4: Remove duplicates and drop unused columns
- Deduplicate rows using `job_id`  
- Drop irrelevant columns like `link` and `date_posted`


In [31]:
# QA check: job_id must exist
assert 'job_id' in jobs_df.columns, "job_id missing"

before = jobs_df.shape[0]
jobs_df = jobs_df.drop_duplicates(subset=["job_id"])
after = jobs_df.shape[0]
print(f"Removed {before-after} duplicate rows using job_id.")

for col in ["link", "date_posted"]:
    if col in jobs_df.columns:
        jobs_df = jobs_df.drop(columns=[col])


Removed 10 duplicate rows using job_id.


## Step 5: Add scrape date
- Insert a fixed scrape date of **2025-09-25**  


In [32]:
jobs_df["scrape_date"] = "2025-09-25"

## Step 6: Clean and enrich fields
- Normalize company names and job titles  
- Translate cleaned titles into English  
- Extract city, canton, remote type, remote flag, and seniority


In [33]:
jobs_df["company_clean"] = jobs_df["company"].apply(clean_company)

jobs_df["title_clean"] = jobs_df["title"].apply(clean_title)

jobs_df["title_en"] = jobs_df["title_clean"].apply(translate_title)

jobs_df[["city","canton","remote_type"]] = jobs_df["location"].apply(lambda x: pd.Series(clean_location(x)))

jobs_df["is_remote"] = jobs_df["remote_type"].eq("Remote")

jobs_df["seniority_level"] = jobs_df["title_en"].apply(extract_seniority)


## Step 7: Save final dataset
- Reorder columns into a consistent structure  
- Save cleaned file to CSV  
- Preview first 10 rows


In [34]:
final_cols = [
    "job_id","title","company","location","scrape_date",
    "company_clean","title_clean","title_en",
    "city","canton","remote_type","is_remote","seniority_level"
]
jobs_df = jobs_df[final_cols]

jobs_df.to_csv(OUT_CSV, index=False, encoding="utf-8")
print(f"✅ Final dataset saved to {OUT_CSV}")
print("Final shape:", jobs_df.shape)
jobs_df.head(10)


✅ Final dataset saved to ../data/Swiss-jobs-cleaned-final.csv
Final shape: (120, 13)


,job_id,title,company,location,scrape_date,company_clean,title_clean,title_en,city,canton,remote_type,is_remote,seniority_level
0,ace6dabef8894c7b,Reference Data Analyst,Eclerx,"Zürich, ZH",2025-09-25,Eclerx,reference data analyst,reference data analyst,Zurich,ZH,Onsite,False,Mid-level
1,c1ccef535fa878f6,Innosight Consulting Analyst - Switzerland,Huron Consulting Group Inc.,"Zürich, ZH",2025-09-25,Huron Consulting Group Inc.,innosight consulting analyst switzerland,innosight consulting analyst switzerland,Zurich,ZH,Onsite,False,Mid-level
2,9f657f8aca704ced,Investment Data Reporting Analyst (Fixed Term ...,LGT Capital Partners,"Pfäffikon, ZH",2025-09-25,LGT Capital Partners,investment data reporting analyst,investment data reporting analyst,Pfaffikon,ZH,Onsite,False,Mid-level
3,10d2123eebee710b,Junior Financial Planning Analyst 80% - 100% (...,Dormakaba Group,"Rümlang, ZH",2025-09-25,Dormakaba Group,junior financial planning analyst,junior financial planning analyst,Rumlang,ZH,Onsite,False,Junior
4,9e554eb40fad3074,"Junior Financial Data Analyst Graduate, Zurich...",Swiss Re,"Zürich, ZH",2025-09-25,Swiss Re,junior financial data analyst graduate zurich,junior financial data analyst graduate zurich,Zurich,ZH,Onsite,False,Entry/Intern
5,d69cec8639002c0e,Business & Data Analyst (A) Migration Project,Bosshard & Partner Unternehmensberatung AG,"Remote in Zürich, ZH",2025-09-25,Bosshard & Partner Unternehmensberatung AG,business & data analyst migration project,business & data analyst migration project,Zurich,ZH,Remote,True,Mid-level
6,26a877169ecbb5a3,(Senior) Data Analyst Online Shop (w/m/d),Digitec Galaxus AG,"Zürich, ZH",2025-09-25,Digitec Galaxus AG,data analyst online shop,data analyst online shop,Zurich,ZH,Onsite,False,Mid-level
7,ea864b60e2a07000,Karrierechancen: Supply Chain Data Analyst (80...,BIOTRONIK,"Bülach, ZH",2025-09-25,BIOTRONIK,karrierechancen supply chain data analyst,Career opportunities supply chain data analyst,Bulach,ZH,Onsite,False,Mid-level
8,561bedede87ff613,(Senior) Data Analyst Online Shop (w/m/d),Galaxus,"Remote in 8005 Zürich, ZH",2025-09-25,Galaxus,data analyst online shop,data analyst online shop,Zurich,ZH,Remote,True,Mid-level
9,c92d1de94ab3b3d8,Data Analyst / Data Scientist für die Lehre (m...,ETH Zürich,"8092 Zürich, ZH",2025-09-25,ETH Zürich,data analyst data scientist für die lehre,data analyst data scientist for teaching,Zurich,ZH,Onsite,False,Mid-level


## Step 8: Manual QA spot check
- Review a random sample of rows to verify cleaning  
- Confirm titles and locations look consistent


In [35]:
sample = jobs_df.sample(6, random_state=42)
sample_preview = sample[['title','title_clean','title_en','location','city','canton']].reset_index(drop=True)
sample_preview.index += 1
sample_preview


,title,title_clean,title_en,location,city,canton
1,Global HR & Compensation Data Analyst,global hr & compensation data analyst,global hr & compensation data analyst,"Pully, VD",Pully,VD
2,Senior Business Analyst,senior business analyst,senior business analyst,"Lausanne, VD",Lausanne,VD
3,"Junior Financial Data Analyst Graduate, Zurich...",junior financial data analyst graduate zurich,junior financial data analyst graduate zurich,"Zürich, ZH",Zurich,ZH
4,Data Analyst,data analyst,data analyst,"Geneva, GE",Geneva,GE
5,Business Analyst – Dashboard Design / PL/SQL /...,business analyst dashboard design pl sql xml,business analyst dashboard design pl sql xml,"Bern, BE",Bern,BE
6,Business Analyst SECLI,business analyst secli,business analyst secli,"Geneva, GE",Geneva,GE


## How I ran this
1. Activated my Conda environment: `conda activate Swiss_projects`  
2. Opened this notebook in Jupyter Lab  
3. Ran all cells in order  
4. Final dataset saved at `../data/Swiss-jobs-cleaned-final.csv`  


## My Closing Notes

While cleaning the dataset, I noticed some encoding issues (e.g., `Zürich` appeared as `ZÃ¼rich`).  
Fixing these was important so that city-level analysis later would not be misleading.  

I chose to standardize job titles in English for consistency, since recruiters and candidates often search roles in English even if postings are multilingual.  
At the same time, I kept company names in their original form to respect brand identity.  

Another important step was creating a `seniority_level` column.  
This makes it easy for recruiters to quickly compare demand for juniors, mid-levels, and seniors across Switzerland.  

By the end of this step, the dataset is structured, recruiter-friendly, and ready for meaningful analysis in the next phase.
